In [8]:
import os
from os import path
import glob
import pandas as pd
import json
from collections import defaultdict

In [9]:
perf_dir = "/home/shtoshni/Research/events/models/perf"
# perf_dir = "/share/data/speech/shtoshni/research/events/models/perf"
slurm_id_list = ["6410467"]  #"6407936"

files = []

for slurm_id in slurm_id_list:
    files.extend(sorted(glob.glob(path.join(perf_dir, slurm_id + "*")), 
                                  key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1])))
    
# files = sorted(files, key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1]))
print(len(files))

9


In [10]:
model_dict_list = []
for file in files:
    model_dict_list.append(json.loads(open(file).read()))

In [11]:
def determine_varying_attributes(model_dict_list, ignore_attribs=['model_dir', 'train', 'test', 'dev', 'pretrained_mention_model',
                                                                  'slurm_id', 'best_model_dir', 'data_dir']):
    attrib_to_vals = defaultdict(set)
    for model_dict in model_dict_list:
        for attrib, val in model_dict.items():
            if attrib in ignore_attribs:
                continue
            else:
                attrib_to_vals[attrib].add(val)
    
    
    varying_attribs = []
    for attrib, vals in attrib_to_vals.items():
        if len(vals) > 1:
            varying_attribs.append(attrib)
            
    return varying_attribs

In [12]:
varying_attribs = determine_varying_attributes(model_dict_list)


In [13]:
perf_df = pd.DataFrame(columns=(varying_attribs))

for model_dict in model_dict_list:
    perf_dict = {}
    for attrib in varying_attribs:
        perf_dict[attrib] = model_dict[attrib]
            
    perf_dict['fscore'] = round(model_dict['dev']['fscore'], 1)
    perf_dict['test_f'] = round(model_dict['test']['fscore'], 1)
                
    perf_df = perf_df.append(perf_dict, ignore_index=True)

In [15]:
idx = perf_df.groupby(['use_srl', 'use_local_attention', 'srl_loss_wt', 'seed', 'doc_proc'])['fscore'].idxmax()
dev_max = perf_df.iloc[idx]
dev_max

KeyError: 'use_local_attention'

In [ ]:
perf_df.groupby(['use_srl', 'use_local_attention', 'srl_loss_wt', 'sample_invalid', 'doc_proc'])[['fscore', 'test_f']].mean()
